In [ ]:
#Install packages

! pip install transformers datasets
! pip install torch
! pip install tensorflow
! pip install tika
! pip install nltk

from transformers import pipeline
import os
import re
import math
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

nltk.download('all')

Clone GitHub repo with all of the data

In [25]:
!git clone https://github.com/bcarter24/testRepo.git

Cloning into 'testRepo'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 302 (delta 9), reused 299 (delta 9), pack-reused 0
Receiving objects: 100% (302/302), 5.95 MiB | 18.75 MiB/s, done.
Resolving deltas: 100% (9/9), done.


Move to data directory

In [26]:
%cd /content/testRepo/data/US\ Federal\ Code\ of\ Law/Cleaner_grouped/

/content/testRepo/data/US Federal Code of Law/Cleaner_grouped


In [28]:
directory1 = "./Title18_Chapter44/"
directory2 = "./Title26_Chapter53/"

document_array = []

%ls
files_in_direc1 = os.listdir(directory1)
print(files_in_direc1)
for file in files_in_direc1:
  file_handle = open(file, 'r')
  this_doc = str( file_handle.read())
  file_handle.close()
  sentence_array = word_tokenize(this_doc,language='english', preserve_line=True)
  document_array.append(sentence_array)
  print('length of sentence array is: ' + str(len(sentence_array)))
  print(sentence_array)
  print('current length of Doc array is: ' + str(len(document_array)))

 testRepo/
 Title18_Chapter44/
'Title18_Chapter44_(27 CFR Regulation 478).pdf'
'Title18_Chapter44_(27 CFR Regulation 478).txt'
 Title26_Chapter53/
'Title26_Chapter53_(27 CFR Regulation 479).pdf'
'Title26_Chapter53_(27 CFR Regulation 479).txt'
['Title18_Chapter44_sections41-60_license.txt', 'Title18_Chapter44_sections121-134_records.txt', 'Title18_Chapter44_sections21-40_administrative.txt', 'Title18_Chapter44_sections71-78_licenseProceedings.txt', 'Title18_Chapter44_sections91-104_business.txt', 'Title18_Chapter44_sections141-153_exemptionsSeizuresForfeitures.txt', 'Title18_Chapter44_sections11-12_definitions.txt', 'Title18_Chapter44_sections171_exportation.txt', 'Title18_Chapter44_sections111-120_Importation.txt']


FileNotFoundError: ignored